In [1]:
# Author : Dheepak Krishnamurthy
# License : BSD 3 Clause
# Title : Custom HoverTool for linked hover tooltips
#         Based on example from Bokeh Docs

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models import HoverTool, CustomJS

output_notebook()

JS_CODE = """
import {HoverToolView, HoverTool} from "models/tools/inspectors/hover_tool"
import * as hittest from "core/hittest"

export class CustomHoverToolView extends HoverToolView
  bind_bokeh_events: () ->
    super()
    for r in @model.computed_renderers
      @listenTo(r.data_source, 'cross-link', @_cross_link)

  _cross_link: (indices, geometry, tool) ->  
    # Probably not necessary
    if ( @.model.id != tool.id ) 
      return
    
    result = hittest.create_hit_test_result()
    result['1d'].indices = indices
    
    r = @model.computed_renderers[0]    
    ds = r.data_source
    @._update(result, @, @plot_view.renderer_views[r.id], ds, {geometry})
      
export class CustomHoverTool extends HoverTool
  default_view: CustomHoverToolView
  type: "CustomHoverTool"
  tool_name: "CustomHover"
  icon: "bk-tool-icon-hover"
"""

class CustomHoverTool(HoverTool):
    __implementation__ = JS_CODE

x = list(range(0, 10))
y0 = [abs(xx) for xx in x]
y1 = y0

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300, title=None)
left.circle('x', 'y0', source=source, radius=.25)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300, title=None)
right.circle('x', 'y1', source=source, radius=.25)

h1 = CustomHoverTool(tooltips='''<div> x: @x </div><div> y: @y0 </div>''')
h2 = CustomHoverTool()

h2.callback = CustomJS(args=dict(hover=h1, source=source), code='''

var indices = cb_data.index['1d'].indices;
source.trigger('cross-link', indices, cb_data.geometry, hover)

''')

left.add_tools(h1)
right.add_tools(h2)

p = gridplot([[left, right]])

show(p)

Loading BokehJS ...